<a href="https://colab.research.google.com/github/leesu00/brainwave/blob/main/%EB%87%8C%ED%8C%8C_%EC%9E%84%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 라이브러리 설치
!pip install mne pyedflib joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 50.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import mne
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import joblib

# .edf 파일을 읽어 데이터프레임으로 변환
def load_and_preprocess_edf(file_path, label):
    raw_data = mne.io.read_raw_edf(file_path, preload=True)
    data, times = raw_data.get_data(return_times=True)
    channel_names = raw_data.ch_names
    df = pd.DataFrame(data.T, columns=channel_names)
    df['time'] = times
    df['label'] = label
    return df

# 부모 폴더 아래의 모든 .edf 파일 읽기
def load_all_edf_files(parent_folder_path, labels):
    dataframes = []
    label_index = 0
    for root, dirs, files in os.walk(parent_folder_path):
        for file in files:
            if file.endswith('.edf'):
                file_path = os.path.join(root, file)
                if label_index < len(labels):
                    df = load_and_preprocess_edf(file_path, labels[label_index])
                    dataframes.append(df)
                    label_index += 1
                else:
                    break  # 레이블보다 파일이 더 많으면 루프 종료
    dataset = pd.concat(dataframes, ignore_index=True)
    return dataset

# 데이터 전처리
def preprocess_data(df):
    X = df.iloc[:, :-2].values  # 'time'과 'label' 열을 제외
    y = df['label'].values
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    num_classes = np.max(y) + 1
    label_map = {index: label for index, label in enumerate(label_encoder.classes_)}
    joblib.dump(scaler, 'scaler.pkl')
    joblib.dump(label_map, 'label_map.pkl')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train = X_train.reshape(-1, X_train.shape[1], 1, 1)
    X_test = X_test.reshape(-1, X_test.shape[1], 1, 1)
    y_train = to_categorical(y_train, num_classes=num_classes)
    y_test = to_categorical(y_test, num_classes=num_classes)
    return X_train, X_test, y_train, y_test, num_classes

# 데이터셋 로드 및 전처리 실행
parent_folder_path = '/content/drive/MyDrive/dman'  # 부모 폴더 경로로 변경
labels = ['Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.', 'Fc4.', 'Fc6.', 'C5..', 'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..', 'Cp5.', 'Cp3.', 'Cp1.'
, 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..'
, 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..'
, 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..']  # 각 파일의 레이블, 실제 레이블로 변경
dataset = load_all_edf_files(parent_folder_path, labels)
X_train, X_test, y_train, y_test, num_classes = preprocess_data(dataset)

Extracting EDF parameters from /content/drive/MyDrive/dman/S104/S104R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 16959  =      0.000 ...   105.994 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S104/S104R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S104/S104R07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S104/S104R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S104/S104R09.edf...
EDF file detected
Setting channel info structure...

<ipython-input-8-bc049c106621>:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_data = mne.io.read_raw_edf(file_path, preload=True)
<ipython-input-8-bc049c106621>:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_data = mne.io.read_raw_edf(file_path, preload=True)
<ipython-input-8-bc049c106621>:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_data = mne.io.read_raw_edf(file_path, preload=True)


Extracting EDF parameters from /content/drive/MyDrive/dman/S100/S100R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S100/S100R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S100/S100R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...


<ipython-input-8-bc049c106621>:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_data = mne.io.read_raw_edf(file_path, preload=True)
<ipython-input-8-bc049c106621>:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_data = mne.io.read_raw_edf(file_path, preload=True)
<ipython-input-8-bc049c106621>:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_data = mne.io.read_raw_edf(file_path, preload=True)


Extracting EDF parameters from /content/drive/MyDrive/dman/S100/S100R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S100/S100R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S100/S100R07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S100/S100R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...


...

Extracting EDF parameters from /content/drive/MyDrive/dman/S100/S100R11.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S100/S100R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S100/S100R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-8-bc049c106621>:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_data = mne.io.read_raw_edf(file_path, preload=True)
<ipython-input-8-bc049c106621>:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_data = mne.io.read_raw_edf(file_path, preload=True)
<ipython-input-8-bc049c106621>:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_data = mne.io.read_raw_edf(file_path, preload=True)


Reading 0 ... 15743  =      0.000 ...   122.992 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S101/S101R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S101/S101R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


<ipython-input-8-bc049c106621>:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_data = mne.io.read_raw_edf(file_path, preload=True)


Extracting EDF parameters from /content/drive/MyDrive/dman/S101/S101R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S101/S101R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S101/S101R13.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S101/S101R09.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /content/drive/MyDrive/dman/S101/S101R05.edf...
EDF file detected
Setting channel info structure..

In [ ]:
def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, kernel_size=(3, 1), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 1)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_model(model, X_train, y_train, X_test, y_test, epochs=10, batch_size=32):
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    model.save('eeg_model.keras')
    return model

input_shape = (X_train.shape[1], 1, 1)
model = build_cnn_model(input_shape, num_classes)
model = train_model(model, X_train, y_train, X_test, y_test)

Epoch 1/10
28104/28104 [==============================] - 184s 7ms/step - loss: 1.2663 - accuracy: 0.5760 - val_loss: 0.9837 - val_accuracy: 0.6678
Epoch 2/10
28104/28104 [==============================] - 174s 6ms/step - loss: 0.8907 - accuracy: 0.6969 - val_loss: 0.8444 - val_accuracy: 0.7133
Epoch 3/10
28104/28104 [==============================] - 171s 6ms/step - loss: 0.7832 - accuracy: 0.7331 - val_loss: 0.7681 - val_accuracy: 0.7381
Epoch 4/10
28104/28104 [==============================] - 179s 6ms/step - loss: 0.7253 - accuracy: 0.7524 - val_loss: 0.7334 - val_accuracy: 0.7512
Epoch 5/10
28104/28104 [==============================] - 182s 6ms/step - loss: 0.6869 - accuracy: 0.7653 - val_loss: 0.6968 - val_accuracy: 0.7625
Epoch 6/10
28104/28104 [==============================] - 182s 6ms/step - loss: 0.6586 - accuracy: 0.7743 - val_loss: 0.6776 - val_accuracy: 0.7692
Epoch 7/10
28104/28104 [==============================] - 182s 6ms/step - loss: 0.6367 - accuracy: 0.7820 - val_

In [ ]:
from keras.models import load_model
import joblib
import mne
import numpy as np

def predict_activity_from_edf(edf_file_path):
    # 학습된 모델, 스케일러, 레이블 맵 로드
    model = load_model('eeg_model.keras')
    scaler = joblib.load('scaler.pkl')
    label_map = joblib.load('label_map.pkl')

    # .edf 파일 읽기
    raw_data = mne.io.read_raw_edf(edf_file_path, preload=True)
    data, times = raw_data.get_data(return_times=True)

    # 데이터 평균 계산 (여러 시점의 데이터를 평균화하여 예측)
    data_mean = np.mean(data, axis=1).reshape(1, -1)

    # 스케일링
    data_scaled = scaler.transform(data_mean)

    # 데이터 형태 변환
    input_data = data_scaled.reshape(-1, data_scaled.shape[1], 1, 1)

    # 예측 수행
    predictions = model.predict(input_data)
    predicted_class = np.argmax(predictions, axis=1)[0]

    # 텍스트로 결과 출력
    predicted_label = label_map[predicted_class]
    return predicted_label

# 예측 예제 사용
example_edf_path = '/content/drive/MyDrive/aman/S001/S001R14.edf'  # 실제 경로로 변경
predicted_activity = predict_activity_from_edf(example_edf_path)
print(f"Predicted activity: {predicted_activity}")

Extracting EDF parameters from /content/drive/MyDrive/aman/S001/S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
1/1 [==============================] - 0s 101ms/step
Predicted activity: Poz.
